# POC — Fast Prompting

Autor: Santiago Curotto

Objetivo: generar copys (texto→texto) y un prompt visual (texto→imagen) con plantilla compacta y evaluación simple.

In [ ]:
import json, random, re
print('OK')

In [ ]:
brief = {
  'rubro':'Gastronomía',
  'producto':'Budines caseros de limón',
  'publico':'Vecinos 20–45',
  'objetivo':'Aumentar pedidos por WhatsApp',
  'estetica':'minimalista'
}

In [ ]:
def gen_copys(b):
  base=[
    f"{b['producto']} con sabor real.",
    f"{b['producto']} como en casa.",
    f"{b['producto']} frescos cada día."
  ]
  outs=[]
  for i,t in enumerate(base,1):
    cta=random.choice(['Escribinos por WhatsApp','Pedilo ahora','Reservá hoy','Consultanos por DM'])
    emoji=' 🍋' if 'budines' in b['producto'].lower() else ''
    outs.append({'variante':i,'copy':f"{t} Confianza local.{emoji} {cta}.", 'cta':cta,'emoji_usado':bool(emoji)})
  return outs

def gen_prompt_visual(b):
  p=f"{b['producto']} en primer plano, fondo liso, luz suave, paleta pastel, 1:1, estilo {b['estetica']}"
  return {'prompt_visual':p,'ajustes':['NightCafe: SD','Steps:30–40','CFG:6–8']}

copys=gen_copys(brief)
prompt_img=gen_prompt_visual(brief)
(copys,prompt_img)

In [ ]:
def score(s):
  cta=any(w in s.lower() for w in ['whatsapp','dm','pedilo','reservá'])
  ben=any(w in s.lower() for w in ['calidad','fresco','confianza','real'])
  n=len(re.findall(r'\w+',s))<=60
  return int(cta)+int(ben)+int(n)

scores=[score(c['copy']) for c in copys]
scores

In [ ]:
# A/B: objetivo original vs cambio mínimo
A=copys
briefB=dict(brief,objetivo='Maximizar clics al perfil')
B=gen_copys(briefB)
res={'brief':brief,'texto_texto':A,'texto_imagen':prompt_img,'ab':{'A':A,'B':B}}
with open('resultados_demo.json','w',encoding='utf-8') as f: json.dump(res,f,ensure_ascii=False,indent=2)
'Guardado resultados_demo.json'

**Conclusión:** Plantillas cortas + variación mínima = resultados consistentes con pocas consultas. Prompt visual utilizable en NightCafe.